In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

2023-08-04 01:19:50.493207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-04 01:19:53.620388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-08-04 01:19:56.387577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-04 01:19:56.736310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-04 01:19:56.736473: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
train_data = pd.read_csv("train.csv")

In [4]:
train_data[train_data["insult"]==1]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
56,0020fd96ed3b8c8b,=Tony Sidaway is obviously a fistfuckee. He lo...,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0


In [5]:
train_data[train_data["id"] == "006b94add72ed61c"]["comment_text"]

176    I think that your a Fagget get a oife and burn...
Name: comment_text, dtype: object

##Preprocessing

In [6]:
from tensorflow.keras.layers import TextVectorization ##This will hwlp in tokeinizing the sentences

In [7]:
x = train_data["comment_text"]
y = train_data[train_data.columns[2:]].values

y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [8]:
MAX_FEATURES = 200000 #Number of words in vocab

In [9]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

2023-08-04 01:19:59.121451: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-04 01:19:59.121618: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-04 01:19:59.121733: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
vectorizer.adapt(x.values)

vectorized_text = vectorizer(x.values)

2023-08-04 01:20:06.381586: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 169146912 exceeds 10% of free system memory.
2023-08-04 01:20:06.381655: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 253720368 exceeds 10% of free system memory.
2023-08-04 01:20:06.722927: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84573456 exceeds 10% of free system memory.
2023-08-04 01:20:06.767076: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 42286728 exceeds 10% of free system memory.
2023-08-04 01:20:06.823807: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 84573456 exceeds 10% of free system memory.


In [11]:
#MCSHBAP: map, cache, shuffle, batch, prefetch
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(32)
dataset = dataset.prefetch(8)#Prevents bottleneck
dataset.as_numpy_iterator().next()

(array([[ 1269,    86,  1540, ...,     0,     0,     0],
        [17003, 28539,   756, ...,     0,     0,     0],
        [  397,    12,   762, ...,     0,     0,     0],
        ...,
        [  187,   118,    13, ...,     0,     0,     0],
        [ 4428,     9,   219, ...,     0,     0,     0],
        [ 3340,    16,     2, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],

In [12]:
x_batch, y_batch = dataset.as_numpy_iterator().next()
x_batch.shape,y_batch.shape

((32, 1800), (32, 6))

In [13]:
train = dataset.take(int(len(dataset)*0.8)+1)
val = dataset.skip(int(len(dataset)*0.8)+1).take(int(len(dataset)*0.2))

##Creating a model


In [14]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [15]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation = "tanh")))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(6, activation="sigmoid"))


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 6)                 774       
                                                                 
Total params: 6466982 (24.67 MB)
Trainable params: 6466982 (24.67 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [17]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
               optimizer="adam",
               metrics=["accuracy"])
stop_early = tf.keras.callbacks.EarlyStopping(monitor="accuracy",patience=3)


In [18]:
model_history = model.fit(x = train,
                            epochs=10,
                            validation_data=val,
                          callbacks = [stop_early]
                            )

Epoch 1/10


2023-08-04 01:20:31.395246: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-04 01:20:32.393086: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1b98bcba70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-04 01:20:32.393137: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-08-04 01:20:33.484695: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-04 01:20:35.429309: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:543] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/

3990/3990 [==============================] - 492s 120ms/step - loss: 0.0633 - accuracy: 0.9803 - val_loss: 0.0437 - val_accuracy: 0.9947
Epoch 2/10
3990/3990 [==============================] - 388s 97ms/step - loss: 0.0445 - accuracy: 0.9686 - val_loss: 0.0393 - val_accuracy: 0.9893
Epoch 3/10
3990/3990 [==============================] - 383s 96ms/step - loss: 0.0391 - accuracy: 0.9767 - val_loss: 0.0339 - val_accuracy: 0.9942
Epoch 4/10
3990/3990 [==============================] - 377s 94ms/step - loss: 0.0340 - accuracy: 0.9780 - val_loss: 0.0298 - val_accuracy: 0.9941


In [19]:
model.save("models")

INFO:tensorflow:Assets written to: models/assets


INFO:tensorflow:Assets written to: models/assets


##Using the model to predict

In [20]:
import tensorflow as tf
model1 = tf.keras.models.load_model('models')

In [21]:
input_text = vectorizer("Ramesh is a gay faggot")

In [22]:
res = model1.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 1s 846ms/step


In [23]:
output = (res>0.5).astype(int)

In [24]:
col = train_data.columns[2:]
col = col.to_numpy()
col.shape

(6,)

In [25]:
count = 0
for index,element in np.ndenumerate(output):
  if(element==1):
    count = count+1
    _,k = index
    print(col[k])
if(count==0):
  print("Not Toxic")

toxic
obscene
insult
